In [1]:
import numpy as np
import os
import tensorflow as tf

n_inputs = 3 # 입력 데이터
n_neurons = 5 # 셀의 가중치 사이즈
tf.reset_default_graph() # 그라프 초기화 ( 변수 생성 )

# 데이터를 받기위해서
X0 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
X1 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
# [0,1,2] 사이즈가 하나의 셀로 입력
# FFNN(feed foward neural network)
# 가중치 사이즈 => 특성을 찾아내는 것
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
# static_rnn : rnn network : 4개의 셀이 연결되면서 메모리 확보
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1],dtype=tf.float32) # 2x4x3 # 배치사이즈*셀수*뉴론수

Y0, Y1 = output_seqs
# 출력, 다음 셀로 전달되는 값 : 마지막 states값 (수평으로 셀을 연결)
init = tf.global_variables_initializer()

X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

C:\Users\ICT01_14\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_14\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_14\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_14\.conda\envs\tf_test\lib\site-packages\tensorflow\

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict={X0: X0_batch, X1: X1_batch})

print("처음 데이터 특성: ", Y0_val, "차수: ",Y0_val.shape)
print("두번째 데이터 특성: ", Y1_val, "차수: ",Y1_val.shape)
# 4x3 => 4x5

처음 데이터 특성:  [[-0.03596    -0.3668864  -0.10731425 -0.8272863   0.8443638 ]
 [ 0.90480936 -0.9192392   0.93030363 -0.99921924  0.9996166 ]
 [ 0.99536276 -0.99238336  0.99790025 -0.9999967   0.99999917]
 [ 0.9982177  -0.32869756  0.99977523 -0.9992169   0.99727964]] 차수:  (4, 5)
두번째 데이터 특성:  [[ 0.9997333  -0.99860656  0.9999742  -0.99999905  0.9999995 ]
 [-0.05964275 -0.9541735  -0.0715422  -0.72048163 -0.05033515]
 [ 0.99583393 -0.99935585  0.99837357 -0.9999455   0.9998699 ]
 [ 0.86382943 -0.983129    0.9299093  -0.9939652   0.96691555]] 차수:  (4, 5)


? 셀이 가지고 있는 가중치 사이즈는 3x5

In [3]:
tf.reset_default_graph()
# 이미지 784 ( 28x28 )
n_steps = 28 # 셀수
n_inputs = 28 # 셀당 인풋사이즈
n_neurons = 150 # 뉴론출력
n_outputs = 10 # 확률 사이즈
learning_rate = 0.001
# 3차원으로
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
# 셀 하나 => FFNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons) # 150
# 셀: 28
# state 마지막 셀의 수평으로 전달된 값
# state는 마지막 셀의  output과 동일
# 28개의 셀이 있는데 마지막 1개의 output을 사용 => many to one
# 감정분류 => 분류기 등으로 사용
# state의 차수 : 150x150
# output의 차수 : latent time 지연시간을 통해서 계산된 셀의 모든 값을 결합 출력 150
# 150*28*150
outputs, states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
# 150개의 특징중에 10개만 추출
# 입력 차수, 출력 차수만 지정하면 자동으로 바이어스 생성
# 가중치 공간을 확보해서
# 150 x 150 => 150 x 10
# dense의 가중치 사이즈 = 150 x 10
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits) # 분류를 위한 미분이 가능한 식
loss = tf.reduce_mean(xentropy) # 배치 사이즈 => 평균을 통해서 구함
# momentum + propgrad
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1) # 가장 큰 값
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
# 원래 이미지 모양으로 학습
X_test = mnist.test.images.reshape((-1,n_steps,n_inputs))
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# n_epochs = 100
# batch_size = 150
# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs): # 100
#         for iteration in range(mnist.train.num_examples // batch_size): # 60000/150
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             # 이미지 사이즈로 생성
#             X_batch = X_batch.reshape((-1, n_steps, n_inputs))
#             sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
#         acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
#         acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test}) # 
#         print(epoch, "Training accuracy: ",acc_train, "Test accuracy: ",acc_test) # 99 # 97

# multi rnn cell

In [6]:
# BasicRNNCell : 기본데이터가 입력
# MultiRNNCell : 수직으로 레이어 구성 : 수직으로 셀이 구성
# dynamic_rnn : 모델
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_neurons = 100
n_layers = 3 # 3대의 멀티 레이어 : 3개의 셀을 생성

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation =tf.nn.relu)
         for layer in range(n_layers)]
# 3개의 셀을 조합해서 MultiRNNCell
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [7]:
# dynamic_rnn : 셀로 입력 되는 데이터의 개수에 맞추어서 셀을 구성
# 나는 학교에 간다. state_rnn 동일한 사이즈 : 큰것 기준 , 작은 것은 padding
# 작으면 입력사이즈를 변동 => 가중치 조절 => 나가는 특징은 일치 (neural수가 같음)
# 3층 셀이 28개가 조성 : 고정 사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# states 가 몇개가 발생하는가?
# 3x150x100(neuron) : 특징을 3번추출
states_concat = tf.concat(axis = 1, values =states) # 150x300 150개의 행이 100개씩 3번 나열됨
logits = tf.layers.dense(states_concat, n_outputs) # 가중치 : 입력, 출력을 넣어주면 알아서 계산해줌 : 300x10
# logits : 150x10 => minibatch 확률값으로
# 원핫인코딩 * log(예측값)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [8]:
# n_epochs = 10
# vatch_size = 150
# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(mnist.train.num_examples // batch_size):
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             X_batch = X_batch.reshape((-1, n_steps, n_inputs))
#             sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
#         acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
#         acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
#         print(epoch, "Training accuracy: ",acc_train, "Test accuracy: ",acc_test)

In [9]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10
# None이 batch_size
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128x28x28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128x10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x10
b = tf.Variable(tf.random_normal([n_class]))  # 10
# 28x128 사이즈의 가중치
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # RNNCell => neurons : 128개의 특징
# cell 수 : 28
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
# 128x28x128, 128x128
# FFNN
outputs = tf.transpose(outputs, [1,0,2]) # 28x128x128
# 셀로
outputs = outputs[-1] # 맨마지막 셀 : 128x128
model = tf.matmul(outputs, W) + b # 128x128 , 가중치 128x10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 60000/128
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch): # 10
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        # python이 계산한 마지막 결과가 저장되는 것이 _
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y:batch_ys})
        total_cost += cost_val
    print('Epoch: ','%04d' % (epoch+1), 'Avg.cost = ', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch:  0001 Avg.cost =  0.475
Epoch:  0002 Avg.cost =  0.233
Epoch:  0003 Avg.cost =  0.185
Epoch:  0004 Avg.cost =  0.156
Epoch:  0005 Avg.cost =  0.146
Epoch:  0006 Avg.cost =  0.124
Epoch:  0007 Avg.cost =  0.121
Epoch:  0008 Avg.cost =  0.111
Epoch:  0009 Avg.cost =  0.114
Epoch:  0010 Avg.cost =  0.102
최적화 완료!


# 문제
1. 테스트 데이터를 이용하여 테스트 하는 회로를 구성하시오.
2. 3개의 셀을 갖는 multi-layer cell로 수정하시오.

In [11]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

# None이 batch_size
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128x28x28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128x10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128*10
b = tf.Variable(tf.random_normal([n_class]))  # 10
# 28x128 사이즈의 가중치

n_layers = 3
# 3개의 셀로 멀티레이어를 생성해도 output은 하나의 셀일 경우와 같다.
# state에서는 3개의 레이어마다 생성되기 때문에
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation =tf.nn.relu)
         for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # RNNCell => neurons : 128개의 특징
# cell 수 : 28
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)
# 128x28x128, 128x128
# FFNN
outputs = tf.transpose(outputs, [1,0,2]) # 28x128x128
# 셀로
outputs = outputs[-1] # 맨마지막 셀 : 128x128
model = tf.matmul(outputs, W) + b # 128x128 , 가중치 128x10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step,n_input)
test_ys = mnist.test.labels

print("정확도:", sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.1099


In [13]:
import numpy as np
values231 = np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])
print("입력데이터의 shape", values231.shape) # 2x3x1
import tensorflow as tf
tf.reset_default_graph()
tf_values231 = tf.constant(values231, dtype=tf.float32)
# 1x100
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=100)
# rnn 모델은 동일 : 망과 망을 연결할 때 (번역망 : 한글)
# latent time 지연시간
outputs,state = tf.nn.dynamic_rnn(cell = lstm_cell,
                             dtype = tf.float32, inputs = tf_values231)

print(state.c.shape) # 100개의 특징 -> 2x100
print(state.h.shape) # 2x100
print("rnn이 출력하는 outputs의 의미",outputs.shape) # 2x3x100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])

입력데이터의 shape (2, 3, 1)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
(2, 100)
(2, 100)
rnn이 출력하는 outputs의 의미 (2, 3, 100)


In [14]:
tf.reset_default_graph()
values = tf.constant(np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
]), dtype = tf.float32)

lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
(output_fw, output_bw), (output_state_fw, output_state_bw) =  tf.nn.bidirectional_dynamic_rnn(
# Multi cell
# 양방향 LSTM
    cell_fw=lstm_cell_fw,
    cell_bw=lstm_cell_bw,
    inputs=values,
    dtype=tf.float32)
print(output_fw.shape)
print(output_bw.shape)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
(2, 3, 100)
(2, 3, 100)


In [15]:
from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
h = [1,0,0,0] # 단어를 원핫 인코딩 : 수치화
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
# 변수 공유를 위해 지정
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape)
    print(states.shape)
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.20142406, -0.3717974 ],
        [ 0.598682  , -0.56830996],
        [-0.15350638, -0.56103694],
        [-0.11054686, -0.13598026],
        [ 0.6371746 , -0.25442633]]], dtype=float32)
(1, 5, 2)
(1, 2)
array([[ 0.6371746 , -0.25442633]], dtype=float32)


# 여기부터ㅡ

In [25]:
tf.reset_default_graph()
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
# a:0, b:1
dic_len = len(num_dic)
# rule base => 자동 툴 => 가중치만 이용
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
# 10x3
# 10x2 target

In [26]:
# 26x26행렬
# 100000000000000000000000
# 010000000000000000000000
# 001000000000000000000000
def make_batch(seq_data) :
    input_batch = []
    target_batch = []
    for seq in seq_data : # 'word'
        input = [num_dic[n] for n in seq[:-1]] # w=22, o = 14, r=17
        target = num_dic[seq[-1]] # d = 3
        input_batch.append(np.eye(dic_len)[input]) # 원핫인코딩
        target_batch.append(target)
    return input_batch, target_batch

In [27]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 10x3x26
Y = tf.placeholder(tf.int32, [None]) # 10x26
#FFNN
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [28]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합을 방지하기 위해 dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5, seed =10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 2레이어로 구성된 멀티셀
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype = tf.float32)
# 10x3x128
# 10x128
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 특성 수 128개

In [29]:
outputs = tf.transpose(outputs, [1,0,2]) # 3*10*128
outputs = outputs[-1] # 맨 뒤의 셀 값 : 10*128

model = tf.matmul(outputs, W) + b # 비선형 회귀방식

#분류
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data) # 단어데이터

for epoch in range(total_epoch) :
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y:target_batch})
    print('Epoch:', '%04d' %(epoch+1), 'cost = ', '{:.6f}'.format(loss))

Epoch: 0001 cost =  3.450627
Epoch: 0002 cost =  2.410267
Epoch: 0003 cost =  1.434291
Epoch: 0004 cost =  1.313289
Epoch: 0005 cost =  0.762723
Epoch: 0006 cost =  0.597162
Epoch: 0007 cost =  0.457204
Epoch: 0008 cost =  0.454105
Epoch: 0009 cost =  0.286228
Epoch: 0010 cost =  0.313508
Epoch: 0011 cost =  0.178165
Epoch: 0012 cost =  0.072517
Epoch: 0013 cost =  0.146923
Epoch: 0014 cost =  0.123103
Epoch: 0015 cost =  0.082210
Epoch: 0016 cost =  0.031887
Epoch: 0017 cost =  0.042517
Epoch: 0018 cost =  0.071473
Epoch: 0019 cost =  0.151240
Epoch: 0020 cost =  0.092791
Epoch: 0021 cost =  0.055310
Epoch: 0022 cost =  0.008208
Epoch: 0023 cost =  0.029702
Epoch: 0024 cost =  0.029801
Epoch: 0025 cost =  0.001981
Epoch: 0026 cost =  0.003138
Epoch: 0027 cost =  0.006098
Epoch: 0028 cost =  0.000399
Epoch: 0029 cost =  0.030323
Epoch: 0030 cost =  0.023374


In [31]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X:input_batch, Y:target_batch})

# 문제 : predict한 결과를 출력하고 accuracy를 출력하시요
# 입력단어와 예측 결과를 나란히 출력해 보시요

In [35]:
print('예측결과 : ', predict, '\n정확도 : ', accuracy_val)

예측결과 :  [ 3  3 15  4  3 11  3  4 18  3] 
정확도 :  1.0


In [49]:
# 단어가 순서대로 입력, 순서대로 예측
predict_words = []
for idx, val in enumerate(seq_data) :
    last_char = char_arr[predict[idx]] # 인덱스가 단어 매핑
    predict_words.append(val[:3] + last_char)
print('입력값', [w[:3] + ' ' for w in seq_data])
print('예측값', predict_words)
print('정확도', accuracy_val)

입력값 ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 1.0
